In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 22a39d96-d905-4578-a497-d81e96c159a6, 3, Finished, Available, Finished)

In [2]:
%%sql
CREATE TABLE if NOT EXISTS Ecommere_Lh.gold_fact_sale (
    Order_ID string ,
    Price float ,
    Quantity float  ,
    Sales float ,
    Discount float  ,
    Profit float    ,
    Shipping_Cost  float   ,
    Order_Date date     ,
    Shipping_Date date  ,
    Product_ID  long    ,
    OrderPriority_ID    long    ,
    ShipMode_ID long    ,
    Customer_ID string  ,
    Order_Year integer  ,
    Order_Month integer ,
    Created_TS timestamp    ,
    Modified_TS timestamp   
)
USING DELTA
PARTITIONED BY(Order_year, Order_Month)

StatementMeta(, 22a39d96-d905-4578-a497-d81e96c159a6, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
max_date = spark.sql("SELECT coalesce(max(Modified_TS),'1900-01-01') from Ecommere_Lh.gold_fact_sale").first()[0]

StatementMeta(, 22a39d96-d905-4578-a497-d81e96c159a6, 5, Finished, Available, Finished)

In [4]:
max_date

StatementMeta(, 22a39d96-d905-4578-a497-d81e96c159a6, 6, Finished, Available, Finished)

'1900-01-01'

In [11]:
df_bronze = spark.sql("""SELECT
bronze_sales.Order_ID   ,
bronze_sales.Sales as Price ,
bronze_sales.Quantity   ,
bronze_sales.Sales * bronze_sales.Quantity as Sales,
bronze_sales.Discount   ,
bronze_sales.Profit     ,
bronze_sales.Shipping_Cost  ,
bronze_sales.Order_Date     ,
bronze_sales.Shipping_Date  ,
gold_product.Product_ID     ,
gold_orderpriority.OrderPriority_ID ,
gold_shipmode.ShipMode_ID   ,
bronze_sales.Customer_ID    ,
Year(Order_Date) as Order_Year  ,
Month(Order_Date) as Order_Month   
from Ecommere_Lh.bronze_sales
INNER JOIN Ecommere_Lh.gold_product on bronze_sales.Product = gold_product.Product AND
                            bronze_sales.Product_Category = gold_product.Product_Category
INNER JOIN Ecommere_Lh.gold_shipmode ON bronze_sales.Ship_Mode = gold_shipmode.Ship_Mode
INNER JOIN Ecommere_Lh.gold_orderpriority on bronze_sales.Order_Priority = gold_orderpriority.Order_Priority

""")

StatementMeta(, 22a39d96-d905-4578-a497-d81e96c159a6, 13, Finished, Available, Finished)

In [12]:
df_bronze.createOrReplaceTempView("ViewFactSale")

StatementMeta(, 22a39d96-d905-4578-a497-d81e96c159a6, 14, Finished, Available, Finished)

In [13]:
%%sql
SELECT * FROM ViewFactSale limit 10

StatementMeta(, 22a39d96-d905-4578-a497-d81e96c159a6, 15, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 15 fields>

In [14]:
%%sql 
merge into Ecommere_Lh.gold_fact_sale as gfs
USING ViewFactSale as vfs
ON gfs.Order_Year = vfs.Order_Year and gfs.Order_Month = vfs.Order_Month and gfs.Order_ID = vfs.Order_ID
WHEN MATCHED THEN
UPDATE SET
gfs.Sales = vfs.Sales   ,
gfs.Price = vfs.Price   ,
gfs.Quantity = vfs.Quantity ,
gfs.Discount = vfs.Discount ,
gfs.Profit  = vfs.Profit    ,
gfs.Shipping_Cost = vfs.Shipping_Cost   ,
gfs.Order_Date = vfs.Order_Date,
gfs.Shipping_Date = vfs.Shipping_Date   ,
gfs.Product_ID = vfs.Product_ID ,
gfs.OrderPriority_ID = vfs.OrderPriority_ID ,
gfs.ShipMode_ID = vfs.ShipMode_ID   ,
gfs.Customer_ID = vfs.Customer_ID   ,
gfs.Modified_TS = CURRENT_TIMESTAMP()

WHEN NOT MATCHED THEN
INSERT
(   
gfs.Order_ID    ,
gfs.Sales   ,
gfs.Price  ,
gfs.Quantity ,
gfs.Discount ,
gfs.Profit    ,
gfs.Shipping_Cost   ,
gfs.Order_Date ,
gfs.Shipping_Date   ,
gfs.Product_ID  ,
gfs.OrderPriority_ID  ,
gfs.ShipMode_ID    ,
gfs.Customer_ID    ,
gfs.Order_Year  ,
gfs.Created_TS  ,
gfs.Modified_TS 

)
VALUES
(
vfs.Order_ID    ,
vfs.Sales   ,
vfs.Price  ,
vfs.Quantity ,
vfs.Discount ,
vfs.Profit    ,
vfs.Shipping_Cost   ,
vfs.Order_Date ,
vfs.Shipping_Date   ,
vfs.Product_ID  ,
vfs.OrderPriority_ID  ,
vfs.ShipMode_ID    ,
vfs.Customer_ID    ,
vfs.Order_Year  ,
CURRENT_TIMESTAMP()  ,
CURRENT_TIMESTAMP()
)

StatementMeta(, 22a39d96-d905-4578-a497-d81e96c159a6, 16, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>